# Export Keras Model to Tensorflow

This notebooks show how to export a keras model to tensorflow.

The Keras model has to use `tf` image dim ordering and `channels_last` image data format.

### Define the model

In [1]:
%matplotlib inline

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU

def create_yolov1_tiny_model(shape):
    model = models.Sequential()

    model.add(layers.Conv2D(16, (3, 3), strides=(1, 1), padding='same', input_shape=shape, name='image'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='valid'))

    model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='valid'))

    model.add(layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='valid'))

    model.add(layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='valid'))

    model.add(layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='valid'))

    model.add(layers.Conv2D(1024, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))

    model.add(layers.Conv2D(1024, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))

    model.add(layers.Conv2D(1024, (3, 3), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.1))

    model.add(layers.Flatten())
    model.add(layers.Dense(256))
    model.add(layers.Dense(4096))
    model.add(LeakyReLU(0.1))
    model.add(layers.Dense(1470, name='prediction'))
    
    return model

Using TensorFlow backend.


### Create model and load weights

In [2]:
from keras import backend as K

K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_last')

model = create_yolov1_tiny_model((448, 448, 3))
model.load_weights('yolo-tiny_tf.weights')

### Export tensorflow model

https://aqibsaeed.github.io/2017-05-02-deploying-tensorflow-model-andorid-device-human-activity-recognition/

In [4]:
import os
import tensorflow as tf

# all new operations will be in test mode from now on
K.set_learning_phase(0)
# serialize the model and get its weights, for quick re-building
config = model.get_config()
weights = model.get_weights()
    
# re-build a model where the learning phase is now hard-coded to 0
new_model = models.Sequential.from_config(config)
new_model.set_weights(weights)

sess = K.get_session()

export_base_path = 'tf-exports'
protobuf_path = os.path.join(export_base_path, 'yolo-tiny-v1.pbtxt')
checkpoint_path = os.path.join(export_base_path, 'yolo-tiny-v1.ckpt')

tf.train.write_graph(sess.graph_def, '.', protobuf_path)

saver = tf.train.Saver()
saver.save(sess, save_path = checkpoint_path)

'tf-exports/yolo-tiny-v1.ckpt'

### Optimize exported model

In [5]:
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

input_node_name = 'image_input'
output_node_name = 'prediction/BiasAdd'

output_frozen_graph_name = os.path.join(export_base_path, 'frozen_yolo.pb')
output_optimized_graph_name = os.path.join(export_base_path, 'optimized_yolo.pb')

freeze_graph.freeze_graph(input_graph = protobuf_path,  input_saver = '',
             input_binary = False, input_checkpoint = checkpoint_path, output_node_names = output_node_name,
             restore_op_name = 'save/restore_all', filename_tensor_name = 'save/Const:0',
             output_graph = output_frozen_graph_name, clear_devices = True, initializer_nodes = '')

input_graph_def = tf.GraphDef()

with tf.gfile.Open(output_frozen_graph_name, 'rb') as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        [input_node_name], 
        [output_node_name],
        tf.float32.as_datatype_enum)

f = tf.gfile.FastGFile(output_optimized_graph_name, 'wb')
f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from tf-exports/yolo-tiny-v1.ckpt
INFO:tensorflow:Froze 24 variables.
Converted 24 variables to const ops.
149 ops in the final graph.


### Test optimized graph

In [6]:
import numpy as np
import tensorflow as tf
import os

from keras.preprocessing.image import load_img

input_node_name = 'image_input'
output_node_name = 'prediction/BiasAdd'
export_base_path = 'tf-exports'
output_optimized_graph_name = os.path.join(export_base_path, 'optimized_yolo.pb')

image_path = '../images/test1.jpg'

image = load_img(image_path, grayscale=False)
original_image = np.array(image)
resized_image = np.array(image.resize((448, 448)))

batch = np.array(resized_image)
batch = 2*(batch/255.) - 1
batch = np.expand_dims(batch, axis=0)

tf.reset_default_graph()

result = None

with tf.Session() as sess:
    with tf.gfile.FastGFile(output_optimized_graph_name, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
        
        predict_op = sess.graph.get_tensor_by_name(output_node_name+':0')
        
        sess.run(tf.global_variables_initializer())
        result = sess.run(predict_op, feed_dict={input_node_name+':0': batch})

In [7]:
def to_bounding_boxes(result, threshold = 0.17):
    grid_size = 7
    num_classes = 20
    num_boxes_per_cell = 2

    predictions = result[:grid_size * grid_size * num_classes]
    assert len(predictions) == 980

    confidences = result[len(predictions) : len(predictions) + grid_size * grid_size * num_boxes_per_cell]
    assert len(confidences) == 98

    coordinates = result[len(predictions) + len(confidences):]
    assert len(coordinates) == 392

    predictions = predictions.reshape([grid_size * grid_size, num_classes])
    confidences = confidences.reshape([grid_size * grid_size, num_boxes_per_cell])
    coordinates = coordinates.reshape([grid_size * grid_size, 2, 4])
    
    boxes = []

    cell_dim = 1. / grid_size

    for cell_i in range(grid_size * grid_size):
        for box_i in range(num_boxes_per_cell):
            grid_row = cell_i // grid_size
            grid_column = cell_i % grid_size

            x, y, width, height = coordinates[cell_i, box_i]

            # We parametrize the bounding box x and y coordinates to be offsets of a particular grid cell location so they are also bounded between 0 and 1
            x = (grid_column * cell_dim) + (x * cell_dim)
            y = (grid_row * cell_dim) + (y * cell_dim)

            # We normalize the bounding box width and height by the image width and height so that they fall between 0 and 1.        
            # This should really be **2, but the original implementation uses **1.8. No idea why though...
            width = width ** 1.8
            height = height ** 1.8

            box_confidence = confidences[cell_i, box_i]

            highest_class_probability_index = np.argmax(predictions[cell_i])
            highest_class_probability = predictions[cell_i, highest_class_probability_index]

            class_confidence = box_confidence * highest_class_probability

            if class_confidence >= threshold:
                boxes.append(np.array([
                    x-(width/2), y-(height/2.), x+(width/2), y+(height/2.),
                    class_confidence, highest_class_probability_index]))
                
    return np.array(boxes)

In [8]:
boxes = to_bounding_boxes(result[0], threshold=0.17)
print(boxes)

[[ 0.66604502  0.56738724  0.7532978   0.6752635   0.27005661  6.        ]
 [ 0.61987297  0.56078984  0.72358974  0.66884433  0.18670553  6.        ]
 [ 0.73898346  0.57076998  0.88162719  0.68145153  0.17469482  6.        ]
 [ 0.82350618  0.53657592  0.98578769  0.72717982  0.42168346  6.        ]]


### Export summary for tensorboard

In [9]:
import tensorflow as tf
from tensorflow.python.platform import gfile

output_log_dir = os.path.join(export_base_path, 'logs')

with tf.Session() as sess:
    model_filename = output_optimized_graph_name
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)

train_writer = tf.summary.FileWriter(output_log_dir)
train_writer.add_graph(sess.graph)

Run `tensorboard --logdir=tf-exports/logs` to visualize the graph structure